In [72]:
import pandas as pd
import numpy as np
import pickle
import spacy
import csv
import fasttext

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

ModuleNotFoundError: No module named 'fasttext'

In [2]:
DATA_PATH = 'Data/'

In [3]:
data = pd.read_csv(DATA_PATH + 'data.csv')

In [67]:
data.code = data.code.apply(lambda x: x.replace(' ', '_'))

In [4]:
data.head()

,code,text,r,counts
0,0705 11 000 0,"Кочанный салат-латук ... свежий, подвергнутый ...",113.5,28
1,3002 15 000 0,Лекарственный препарат ... в виде раствора для...,1482.0,35
2,0705 11 000 0,"Смесь из овощей «...», состоящая из кочанного ...",113.5,28
3,3002 15 000 0,Лекарственный препарат ... в виде раствора для...,1482.0,35
4,0705 11 000 0,"Смесь из овощей «...», состоящая из кочанного ...",113.5,28


In [26]:
!pip install -U spacy[cuda113]

     ---------------------------------------- 11.6/11.6 MB 9.8 MB/s eta 0:00:00
     ---------------------------------------- 56.9/56.9 MB 8.2 MB/s eta 0:00:00


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\nikon\\Anaconda3\\Lib\\site-packages\\cupy\\_core\\core.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [58]:
nlp = spacy.load(
    "ru_core_news_lg", disable=["tagger", "parser", "ner", "attribute_ruler"]
)


def lemmatize_pipe(doc):
    vector_list = [
        tok.lemma_
        for tok in doc
        if tok.is_alpha and not tok.is_punct and not tok.is_stop
    ]
    return vector_list


def preprocess_pipe(texts):
    preproc_pipe = []
    for doc in nlp.pipe(texts):
        preproc_pipe.append(lemmatize_pipe(doc))
    return pd.Series(preproc_pipe)

In [61]:
tokens = preprocess_pipe(data.text)


In [65]:
data['preproc_text'] = tokens

In [68]:
data.head()

,code,text,r,counts,preproc_text
0,0705_11_000_0,"Кочанный салат-латук ... свежий, подвергнутый ...",113.5,28,"[кочанный, салат, латук, свежий, подвергнуть, ..."
1,3002_15_000_0,Лекарственный препарат ... в виде раствора для...,1482.0,35,"[лекарственный, препарат, вид, раствор, подкож..."
2,0705_11_000_0,"Смесь из овощей «...», состоящая из кочанного ...",113.5,28,"[смесь, овощ, состоять, кочанный, салат, латук..."
3,3002_15_000_0,Лекарственный препарат ... в виде раствора для...,1482.0,35,"[лекарственный, препарат, вид, раствор, подкож..."
4,0705_11_000_0,"Смесь из овощей «...», состоящая из кочанного ...",113.5,28,"[смесь, овощ, состоять, кочанный, салат, латук..."


In [70]:
train, test = train_test_split(data)

In [ ]:
train

,code,text,r,counts
1736,8479_82_000_0,оборудование взвешивание смешивание материал п...,10485.5,90
2799,8474_90_900_0,клин неподвижный щека использоваться крепление...,10025.0,97
4014,8477_20_000_0,экструзионный линия производство лист полиэтил...,10149.5,68
259,8436_29_000_0,автоматический линия подача корм система поени...,8166.0,47
3723,8477_20_000_0,оборудование представлять себя автоматизирован...,10149.5,68
...,...,...,...,...
1245,8471_49_000_0,персональный компьютер предназначить автоматич...,9512.0,35
2821,8421_21_000_9,картридж фильтровать элемент тип xl вид пласти...,6607.0,113
3608,8419_89_989_0,изделие предназначить подключение высокоэффект...,6376.5,200
1492,8474_90_900_0,щека дробящий сбор щековый дробилка представля...,10025.0,97


In [ ]:
#train.text = train.text.apply(lambda x: ' '.join(simple_preprocess(x)))
#test.text = test.text.apply(lambda x: ' '.join(simple_preprocess(x)))

train.code = train.code.apply(lambda x: '__label__' + str(x))
test.code = test.code.apply(lambda x: '__label__' + str(x))

In [ ]:
# Saving the CSV file as a text file to train/test the classifier
train[['text', 'code']].to_csv('train_fasttext_baseline.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

test[['text', 'code']].to_csv('test_fasttext_baseline.txt', 
                                               index = False, 
                                               sep = ' ',
                                               header = None, 
                                               quoting = csv.QUOTE_NONE, 
                                               quotechar = "", 
                                               escapechar = " ")

In [ ]:
# Training the fastText classifier
model = fasttext.train_supervised('/content/train_fasttext_baseline.txt',
                                  lr=0.5,                # learning rate [0.1]
                                  dim=36,               # size of word vectors [100]
                                  ws=10,                # size of the context window [5]
                                  epoch=70,             # number of epochs [5]
                                  neg=250,               # number of negatives sampled [5]
                                  wordNgrams=3)

In [ ]:
aaa = model.predict(aa, k=5)[0]

In [ ]:
for i in aaa:
    print(i[9:])

3004_50_000_2
3002_15_000_0
3004_39_000_1
3307_90_000_8
3004_20_000_2


In [ ]:
_, precision, recall = model.test('/content/train_fasttext_baseline.txt') 
leaf_F1 = (2*precision*recall) / (precision+recall)

In [ ]:
leaf_F1

1.0

In [ ]:
_, precision, recall = model.test('/content/test_fasttext_baseline.txt')                      
leaf_F1 = (2*precision*recall) / (precision+recall)

In [ ]:
leaf_F1

0.6599516518936341

0.5825946817082998

0.6390008058017728 - кастомный препроцессинг

0.6591458501208702 lr 0.5, dim 60, ws 10, neg 100

In [ ]:
pred = test.copy()

In [ ]:
pred.r = pred.text

In [ ]:
pred.r = pred.r.apply(lambda x: model.predict(x)[0][0])

In [ ]:
pred

,code,text,r,counts
4737,__label__8438_20_000_0,автоматизированный линия представлять себя ком...,__label__8438_20_000_0,29
373,__label__8419_89_989_0,ава автоклав стерилизация предназначить стерил...,__label__8419_89_989_0,200
2300,__label__3004_20_000_2,азимед таблетка покрыть плёночный оболочка све...,__label__3004_90_000_2,71
2188,__label__8434_20_000_0,линия производство сыр состоять следующий комп...,__label__8474_80_908_0,24
3550,__label__8422_40_000_8,линия мойка упаковка предназначить упаковка ме...,__label__8424_89_000_9,58
...,...,...,...,...
4439,__label__3004_90_000_2,таблетка покрыть плёночный оболочка мг таблетк...,__label__3004_90_000_2,794
2993,__label__8474_90_900_0,статор флотационный машина изделие полиуретан ...,__label__8474_90_900_0,97
4780,__label__3004_90_000_2,арёлтый таблетка покрыть плёночный оболочка ко...,__label__3004_90_000_2,794
748,__label__3004_90_000_2,тиоцетам таблетка покрыть плёночный оболочка ж...,__label__3004_90_000_2,794


In [ ]:
accuracy_score(pred.code, pred.r)

0.6599516518936341

In [ ]:
model.

[(0.9591032266616821, 'шпур'),
 (0.9327155351638794, 'кевларов'),
 (0.9271788001060486, 'огнев'),
 (0.9103687405586243, 'присоска'),
 (0.8937010169029236, 'зазубрина'),
 (0.8914391994476318, 'простенок'),
 (0.8911378979682922, 'ts'),
 (0.8880701065063477, 'замковый'),
 (0.8787413835525513, 'траверс'),
 (0.8765773177146912, 'воздухоотвод')]

In [ ]:
roc_auc_score(pred.code, pred.r)

ValueError: ignored

In [ ]:
def predict(document):
    return model.predict(document, k=5)

In [ ]:
test['ans'] = test.text.apply(lambda text: predict(text)[0])

In [ ]:
test = test.reset_index()
test = test.drop(columns=['index'])

In [ ]:
test = test.drop(columns=['index', 'level_0'])

In [ ]:
test

,code,text,r,counts,ans
0,__label__8438_20_000_0,автоматизированный линия представлять себя ком...,8468.0,29,"(__label__8438_20_000_0, __label__8434_20_000_..."
1,__label__8419_89_989_0,ава автоклав стерилизация предназначить стерил...,6376.5,200,"(__label__8419_89_989_0, __label__8419_50_000_..."
2,__label__3004_20_000_2,азимед таблетка покрыть плёночный оболочка све...,1596.0,71,"(__label__3004_90_000_2, __label__3004_20_000_..."
3,__label__8434_20_000_0,линия производство сыр состоять следующий комп...,8089.5,24,"(__label__8474_80_908_0, __label__8438_20_000_..."
4,__label__8422_40_000_8,линия мойка упаковка предназначить упаковка ме...,6988.5,58,"(__label__8424_89_000_9, __label__8479_81_000_..."
...,...,...,...,...,...
1236,__label__3004_90_000_2,таблетка покрыть плёночный оболочка мг таблетк...,2220.5,794,"(__label__3004_90_000_2, __label__3004_39_000_..."
1237,__label__8474_90_900_0,статор флотационный машина изделие полиуретан ...,10025.0,97,"(__label__8474_90_900_0, __label__8454_90_000_..."
1238,__label__3004_90_000_2,арёлтый таблетка покрыть плёночный оболочка ко...,2220.5,794,"(__label__3004_90_000_2, __label__3004_50_000_..."
1239,__label__3004_90_000_2,тиоцетам таблетка покрыть плёночный оболочка ж...,2220.5,794,"(__label__3004_90_000_2, __label__3004_50_000_..."


In [ ]:
test.ans[3550]

('__label__8424_89_000_9',
 '__label__8479_81_000_0',
 '__label__8422_30_000_8',
 '__label__8422_20_000_9',
 '__label__8438_50_000_0')

In [ ]:
test.ans.tolist()

In [ ]:
test[['t1','t2','t3','t4','t5']] = pd.DataFrame(test.ans.tolist(), index= test.index)

In [ ]:
test

,code,text,r,counts,ans,t1,t2,t3,t4,t5
0,__label__8438_20_000_0,автоматизированный линия представлять себя ком...,8468.0,29,"(__label__8438_20_000_0, __label__8434_20_000_...",__label__8438_20_000_0,__label__8434_20_000_0,__label__8479_20_000_0,__label__8419_89_989_0,__label__8418_69_000_8
1,__label__8419_89_989_0,ава автоклав стерилизация предназначить стерил...,6376.5,200,"(__label__8419_89_989_0, __label__8419_50_000_...",__label__8419_89_989_0,__label__8419_50_000_0,__label__8418_69_000_8,__label__8419_90_850_9,__label__7309_00_590_0
2,__label__3004_20_000_2,азимед таблетка покрыть плёночный оболочка све...,1596.0,71,"(__label__3004_90_000_2, __label__3004_20_000_...",__label__3004_90_000_2,__label__3004_20_000_2,__label__3004_50_000_2,__label__3004_49_000_8,__label__3004_39_000_1
3,__label__8434_20_000_0,линия производство сыр состоять следующий комп...,8089.5,24,"(__label__8474_80_908_0, __label__8438_20_000_...",__label__8474_80_908_0,__label__8438_20_000_0,__label__8474_39_000_9,__label__8474_80_108_0,__label__8434_20_000_0
4,__label__8422_40_000_8,линия мойка упаковка предназначить упаковка ме...,6988.5,58,"(__label__8424_89_000_9, __label__8479_81_000_...",__label__8424_89_000_9,__label__8479_81_000_0,__label__8422_30_000_8,__label__8422_20_000_9,__label__8438_50_000_0
...,...,...,...,...,...,...,...,...,...,...
1236,__label__3004_90_000_2,таблетка покрыть плёночный оболочка мг таблетк...,2220.5,794,"(__label__3004_90_000_2, __label__3004_39_000_...",__label__3004_90_000_2,__label__3004_39_000_1,__label__3004_20_000_2,__label__3004_50_000_2,__label__3004_49_000_8
1237,__label__8474_90_900_0,статор флотационный машина изделие полиуретан ...,10025.0,97,"(__label__8474_90_900_0, __label__8454_90_000_...",__label__8474_90_900_0,__label__8454_90_000_0,__label__8480_60_000_0,__label__8431_49_200_0,__label__6307_90_980_0
1238,__label__3004_90_000_2,арёлтый таблетка покрыть плёночный оболочка ко...,2220.5,794,"(__label__3004_90_000_2, __label__3004_50_000_...",__label__3004_90_000_2,__label__3004_50_000_2,__label__3004_39_000_1,__label__3004_49_000_8,__label__8517_62_000_3
1239,__label__3004_90_000_2,тиоцетам таблетка покрыть плёночный оболочка ж...,2220.5,794,"(__label__3004_90_000_2, __label__3004_50_000_...",__label__3004_90_000_2,__label__3004_50_000_2,__label__3004_49_000_8,__label__3004_20_000_2,__label__3004_39_000_1


In [ ]:
data_shape = test.shape[0]

In [ ]:
lvl_1 = test[test.code == test.t1].shape[0]

In [ ]:
lvl_2 = test[(test.code == test.t2)].shape[0]

In [ ]:
lvl_3 = test[(test.code == test.t3)].shape[0]

In [ ]:
lvl_4 = test[(test.code == test.t4)].shape[0]

In [ ]:
lvl_5 = test[(test.code == test.t5)].shape[0]

In [ ]:
# accuracy top 3
(lvl_1 + lvl_2 + lvl_3) / data_shape

0.8364222401289283

In [ ]:
# accuracy top 5
(lvl_1 + lvl_2 + lvl_3 + lvl_4 + lvl_5) / data_shape

0.8896051571313457

In [ ]:
all_dataset = pd.concat([test[['code', 'text']], train[['code', 'text']]]).reset_index()

In [ ]:
all_dataset

,index,code,text
0,0,__label__8438_20_000_0,автоматизированный линия представлять себя ком...
1,1,__label__8419_89_989_0,ава автоклав стерилизация предназначить стерил...
2,2,__label__3004_20_000_2,азимед таблетка покрыть плёночный оболочка све...
3,3,__label__8434_20_000_0,линия производство сыр состоять следующий комп...
4,4,__label__8422_40_000_8,линия мойка упаковка предназначить упаковка ме...
...,...,...,...
4957,1245,__label__8471_49_000_0,персональный компьютер предназначить автоматич...
4958,2821,__label__8421_21_000_9,картридж фильтровать элемент тип xl вид пласти...
4959,3608,__label__8419_89_989_0,изделие предназначить подключение высокоэффект...
4960,1492,__label__8474_90_900_0,щека дробящий сбор щековый дробилка представля...


In [ ]:
# Saving the CSV file as a text file to train/test the classifier
all_dataset[['text', 'code']].to_csv('all_data_top5.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

In [ ]:
# Training the fastText classifier
model_all_data = fasttext.train_supervised('/content/all_data_top5.txt',
                                  lr=0.4,                # learning rate [0.1]
                                  dim=36,               # size of word vectors [100]
                                  ws=10,                # size of the context window [5]
                                  epoch=70,             # number of epochs [5]
                                  neg=250,               # number of negatives sampled [5]
                                  wordNgrams=3)

In [ ]:
_, precision, recall = model_all_data.test('/content/all_data_top5.txt') 
leaf_F1 = (2*precision*recall) / (precision+recall)
leaf_F1

0.9997984683595325

In [ ]:
model_all_data.save_model('FastText_top5.fasttext_model')

In [ ]:
model.predict('/content/test_fasttext_baseline.txt')

(('__label__3926_90_970_7',), array([1.00000978]))

In [ ]:
t = 'Клостилбегит таблетки 50мг №10, круглые, плоские таблетки с фаской, с гравировкой в виде на одной стороне, белого, желтовато-белого, или серовато-белого цвета; без или почти без запаха. Одна таблетка содержит активное вещество - кломифена цитрат 50 мг, вспомогательные вещества: лактозы моногидрат, крахмал картофельный, тальк Е-553b, желатин, магния стеарат Е-572, кислота стеариновая Е-570.Половые гормоны и модуляторы половой системы. Гонадотропины и другие стимуляторы овуляции. Показания к применению: индукция овуляции, нарушения овуляции при недостаточности функции гипоталамуса, вторичная аменорея различного происхождения, дисфункциональные маточные кровотечения, олигоменорея, галакторея неопухолевого происхождения, синдром Штейна-Левенталя, синдром Киари-Фроммеля, олигоспермия. Форма выпуска и упаковки: по 10 таблеток во флаконе из коричневого стекла. Флакон с инструкцией по применению помещен в картонную коробку.'

In [ ]:
t_preproc = ' '.join(simple_preprocess(t))

In [ ]:
model.predict(t_preproc)[0][0][9:]

'3004_90_000_2'

In [ ]:
import pickle

In [ ]:
with open('model.pickle', 'wb') as f:
    pickle.dump(model, f)

TypeError: ignored

In [ ]:
model.save_model('FastText_baseline.fasttext_model')

In [ ]:
model2 = fasttext.load_model('/content/FastText_baseline.fasttext_model')

In [ ]:
model2.predict(t_preproc)[0][0][9:]

'3214_90_000_9'

In [ ]:
train

,code,text,r,counts
1089,__label__3004_50_000_2,торговое наименование международное непатентов...,1771.0,91
3401,__label__3004_90_000_2,состав одна таблетка содержит активное веществ...,2220.5,794
1727,__label__8438_10_100_0,машина предназначена для нарезки хлебобулочных...,8402.0,83
2869,__label__3926_90_970_7,накладки для кормления предназначены для кормл...,3768.5,34
3464,__label__3004_90_000_2,таблетки мг белые круглые двояковыпуклые табле...,2220.5,794
...,...,...,...,...
4499,__label__3004_90_000_2,клималанин таблетки белого цвета формы со скош...,2220.5,794
148,__label__8422_30_000_8,оборудование представляет собой автоматическую...,6877.0,157
4269,__label__8419_89_989_0,камера для дефростации продуктов блоках предна...,6376.5,200
3750,__label__2309_90_960_9,кормовая добавка виде сыпучего порошка от бело...,720.0,81


# train on all dataset

In [ ]:
data

,code,text,r,counts
0,0705_11_000_0,"Кочанный салат-латук ... свежий, подвергнутый ...",113.5,28
1,3002_15_000_0,Лекарственный препарат ... в виде раствора для...,1482.0,35
2,0705_11_000_0,"Смесь из овощей «...», состоящая из кочанного ...",113.5,28
3,3002_15_000_0,Лекарственный препарат ... в виде раствора для...,1482.0,35
4,0705_11_000_0,"Смесь из овощей «...», состоящая из кочанного ...",113.5,28
...,...,...,...,...
4957,3004_50_000_2,Товар Представляет Собой Шипучие Таблетки Акти...,1771.0,91
4958,3004_39_000_1,"Лориста®, Таблетки, Покрытые Пленочной Оболочк...",1671.5,38
4959,3004_50_000_2,Товар Представляет Собой Шипучие Таблетки Акти...,1771.0,91
4960,3004_39_000_1,"Лориста® 50 Мг, Таблетки, Покрытые Пленочной О...",1671.5,38


In [ ]:
data_all = data.copy()

In [ ]:
data_all.text = data_all.text.apply(lambda x: ' '.join(simple_preprocess(x)))

data_all.code = data_all.code.apply(lambda x: '__label__' + str(x).replace(" ", '_'))

In [ ]:
data_all

,code,text,r,counts
0,__label__0705_11_000_0,кочанный салат латук свежий подвергнутый ручно...,113.5,28
1,__label__3002_15_000_0,лекарственный препарат виде раствора для подко...,1482.0,35
2,__label__0705_11_000_0,смесь из овощей состоящая из кочанного салата ...,113.5,28
3,__label__3002_15_000_0,лекарственный препарат виде раствора для подко...,1482.0,35
4,__label__0705_11_000_0,смесь из овощей состоящая из кочанного салата ...,113.5,28
...,...,...,...,...
4957,__label__3004_50_000_2,товар представляет собой шипучие таблетки акти...,1771.0,91
4958,__label__3004_39_000_1,лориста таблетки покрытые пленочной оболочкой ...,1671.5,38
4959,__label__3004_50_000_2,товар представляет собой шипучие таблетки акти...,1771.0,91
4960,__label__3004_39_000_1,лориста мг таблетки покрытые пленочной оболочк...,1671.5,38


In [ ]:
data_all[['text', 'code']].to_csv('data_all_fasttext_baseline.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

In [ ]:
model_all = fasttext.train_supervised('/content/data_all_fasttext_baseline.txt',
                                  lr=0.3,                # learning rate [0.1]
                                  dim=100,               # size of word vectors [100]
                                  ws=10,                # size of the context window [5]
                                  epoch=40,             # number of epochs [5]
                                  neg=150,               # number of negatives sampled [5]
                                  wordNgrams=3)

In [ ]:
model_all.save_model('All_data_FastText_baseline.fasttext_model')

In [ ]:
t = '''
Механическое распылительное устройство для каткрекинга, позволяет снизить процесс осушения газа и нефтяной эмульсии, увеличивая процесс переработки нефти.
Масло подается к райзеру (колонне) реактора, после чего разделяется на равное количество на каждое распылительное устройство. Поданное масло мелко распыленное масло перемешивается с дисперсионным паром в механическом распылительном устройстве, после чего впрыскивается в райзер (колонну).
Малые капли сырья контактируют с горячим регенерированным катализатором при параллельном потоке и испаряются. Выпаренное масло перемешивается с частичками катализатора и происходит процесс крекинга в легкие продукты крекинга наряду с мазутом, коксом и газом. Пар продуктов (сжиженный углеводородный газ, газолин, ЛРГ) поднимается по колонне (райдеру) во время трансфера катализатора).
Специально сконструированная система механического впрыскивания минимизирует образование кокса, газа и сжиженного углеводородного газа.'''

In [ ]:
t_preproc = ' '.join(simple_preprocess(t))

In [ ]:
t_preproc

'механическое распылительное устройство для каткрекинга позволяет снизить процесс осушения газа нефтяной эмульсии увеличивая процесс переработки нефти масло подается райзеру колонне реактора после чего разделяется на равное количество на каждое распылительное устройство поданное масло мелко распыленное масло перемешивается дисперсионным паром механическом распылительном устройстве после чего впрыскивается райзер колонну малые капли сырья контактируют горячим катализатором при параллельном потоке испаряются выпаренное масло перемешивается частичками катализатора происходит процесс крекинга легкие продукты крекинга наряду мазутом коксом газом пар продуктов сжиженный углеводородный газ газолин лрг поднимается по колонне райдеру во время трансфера катализатора специально система механического впрыскивания минимизирует образование кокса газа сжиженного углеводородного газа'

In [ ]:
model_all.predict(t_preproc)[0][0][9:]

'8419_89_989_0'

In [ ]:
_, precision, recall = model_all.test('/content/data_all_fasttext_baseline.txt') 
leaf_F1 = (2*precision*recall) / (precision+recall)

In [ ]:
leaf_F1

0.7553405884723902

In [ ]:
n = 412

In [ ]:
data.text[n]

'Механическое распылительное устройство для каткрекинга, позволяет снизить процесс осушения газа и нефтяной эмульсии, увеличивая процесс переработки нефти.\nМасло подается к райзеру (колонне) реактора, после чего разделяется на равное количество на каждое распылительное устройство. Поданное масло мелко распыленное масло перемешивается с дисперсионным паром в механическом распылительном устройстве, после чего впрыскивается в райзер (колонну).\nМалые капли сырья контактируют с горячим регенерированным катализатором при параллельном потоке и испаряются. Выпаренное масло перемешивается с частичками катализатора и происходит процесс крекинга в легкие продукты крекинга наряду с мазутом, коксом и газом. Пар продуктов (сжиженный углеводородный газ, газолин, ЛРГ) поднимается по колонне (райдеру) во время трансфера катализатора).\nСпециально сконструированная система механического впрыскивания минимизирует образование кокса, газа и сжиженного углеводородного газа.'

In [ ]:
data.code[n]

'8424 89 000 9'

# spacy preproc

In [ ]:
import spacy #load spacy

In [ ]:
data_spacy = data.copy()

In [ ]:
data_spacy

,code,text,r,counts
0,0705 11 000 0,"Кочанный салат-латук ... свежий, подвергнутый ...",113.5,28
1,3002 15 000 0,Лекарственный препарат ... в виде раствора для...,1482.0,35
2,0705 11 000 0,"Смесь из овощей «...», состоящая из кочанного ...",113.5,28
3,3002 15 000 0,Лекарственный препарат ... в виде раствора для...,1482.0,35
4,0705 11 000 0,"Смесь из овощей «...», состоящая из кочанного ...",113.5,28
...,...,...,...,...
4957,3004 50 000 2,Товар Представляет Собой Шипучие Таблетки Акти...,1771.0,91
4958,3004 39 000 1,"Лориста®, Таблетки, Покрытые Пленочной Оболочк...",1671.5,38
4959,3004 50 000 2,Товар Представляет Собой Шипучие Таблетки Акти...,1771.0,91
4960,3004 39 000 1,"Лориста® 50 Мг, Таблетки, Покрытые Пленочной О...",1671.5,38


In [ ]:
!pip install spacy -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.2 MB 5.1 MB/s 
     |████████████████████████████████| 660 kB 39.4 MB/s 
     |████████████████████████████████| 10.1 MB 32.8 MB/s 
     |████████████████████████████████| 457 kB 38.3 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 181 kB 42.7 MB/s 
     |████████████████████████████████| 58 kB 5.4 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srs

In [ ]:
!python -m spacy download ru_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 513.4 MB 5.7 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [ ]:
nlp = spacy.load("ru_core_news_lg", disable=['parser', 'tagger', 'ner'])
stops = stopwords.words("russian")

def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = str(word.lemma).strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)


data_spacy['Text_After_Clean'] = data_spacy['text'].apply(normalize, lowercase=True, remove_stopwords=True)

In [ ]:
data_spacy.Text_After_Clean[0]

'12415946700229140116 6020021547390382609 9153284864653046197 8688639969922810138 10875615029400813363 3734125755010985993 2593208677638477497 5377038038267841909 2343356656886760454 1939769312754094652 2593208677638477497 8575667991819561323 9209225331278267973 241599264570149814 2593208677638477497 8575667991819561323 3840634852512275221 2593208677638477497 17163931785380153171 13856456111292828389 16191904166009283104 17127319626481770500 12638816674900267446 6012636577876033220 1183163957922634920 13856456111292828389 2491473056721672295 2090661578966068036 9153284864653046197 10999827425508017904 3771369677110732816 2593208677638477497 9736222341475624889 5087266072664039774 9153284864653046197 12136287192145005166 3771369677110732816 3842344029291005339 2593208677638477497 12058549539613726169 2593208677638477497 5629484814614706838 6528743480287290748 15063453645533333295 2593208677638477497 8575667991819561323 3795023386666704204 14519607967332386503 12646065887601541794 531971